In [5]:
import pandas as pd
import io
import re


In [6]:
# Loading the CSV data
csv_data = "raw_data.csv"
df = pd.read_csv(csv_data)

# Cleaning the 'hour' column to extract only HH:MM
def clean_hour(hour_str):
    if isinstance(hour_str, str):
        # Extract HH:MM using regex to match time at the start
        match = re.match(r'(\d{2}:\d{2})', hour_str)
        return match.group(1) if match else hour_str
    return hour_str

df['hour'] = df['hour'].apply(clean_hour)

# Splitting date and time into separate columns
# Extract year, month, day from 'date'
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Extract hour and minute from 'hour'
df['time'] = pd.to_datetime(df['hour'], format='%H:%M', errors='coerce')
df['hour_value'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute

# Dropping temporary 'time' and original 'date' and 'hour' columns
df = df.drop(columns=['date', 'hour', 'time'])

# Filling missing values in 'visibility'
# Replace 'N/A' with None
df['visibility'] = df['visibility'].replace('N/A', None)
# Convert to numeric, coercing errors to NaN
df['visibility'] = pd.to_numeric(df['visibility'], errors='coerce')
# Use forward fill for missing values
df['visibility'] = df['visibility'].fillna(method='ffill')
# Alternatively, use interpolation for smoother filling (uncomment if preferred)
# df['visibility'] = df['visibility'].interpolate(method='linear')

# Reordering columns for clarity
df = df[['year', 'month', 'day', 'hour_value', 'minute', 'weather', 'temperature', 'wind_speed', 
         'wind_angle', 'humidity', 'pressure', 'visibility']]
df.head()

C:\Users\nptha\AppData\Local\Temp\ipykernel_37424\425859679.py:36: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['visibility'] = df['visibility'].fillna(method='ffill')


,year,month,day,hour_value,minute,weather,temperature,wind_speed,wind_angle,humidity,pressure,visibility
0,2020,1,1,0,0,Passing clouds.,24.0,11.0,90.0,89.0,1021.0,6.0
1,2020,1,1,0,30,Passing clouds.,25.0,9.0,90.0,83.0,1021.0,6.0
2,2020,1,1,1,0,Passing clouds.,25.0,9.0,90.0,83.0,1020.0,6.0
3,2020,1,1,1,30,Passing clouds.,24.0,9.0,90.0,89.0,1020.0,6.0
4,2020,1,1,2,0,Passing clouds.,25.0,6.0,100.0,89.0,1020.0,6.0


In [8]:
def simplify_weather(desc):
    desc = str(desc).lower()
    
    if any(keyword in desc for keyword in ["rain", "shower", "thunderstorm", "sprinkle", "storm"]):
        return "Rainy"
    elif any(keyword in desc for keyword in ["cloud", "overcast", "fog", "haze", "more clouds", "passing clouds", "scattered clouds", "broken clouds"]):
        return "Cloudy"
    elif any(keyword in desc for keyword in ["clear", "sunny", "hot", "warm", "mild"]):
        return "Clear"
    else:
        return "Unknown"

# Áp dụng vào DataFrame
df['weather'] = df['weather'].apply(simplify_weather)

# Kiểm tra kết quả phân phối
print(df['weather'].value_counts())


weather
Cloudy     70231
Rainy      10055
Clear       7448
Unknown       29
Name: count, dtype: int64


In [ ]:
# Saving the processed data to a new CSV
output = "Clean_data/clean_data.csv"
# Assuming a function to save the file exists, e.g., 
df.to_csv(output, index=False)
df.head()

,year,month,day,hour_value,minute,weather,temperature,wind_speed,wind_angle,humidity,pressure,visibility
0,2020,1,1,0,0,Cloudy,24.0,11.0,90.0,89.0,1021.0,6.0
1,2020,1,1,0,30,Cloudy,25.0,9.0,90.0,83.0,1021.0,6.0
2,2020,1,1,1,0,Cloudy,25.0,9.0,90.0,83.0,1020.0,6.0
3,2020,1,1,1,30,Cloudy,24.0,9.0,90.0,89.0,1020.0,6.0
4,2020,1,1,2,0,Cloudy,25.0,6.0,100.0,89.0,1020.0,6.0
